In [ ]:
import os
import sys
from pathlib import Path

# Navigate to project root (equivalent to cd ..)
project_dir = Path(__file__).parent.parent if '__file__' in globals() else Path.cwd().parent
os.chdir(project_dir)

# Add src directory to Python path for imports
src_dir = project_dir / "src"
if str(src_dir) not in sys.path:
    sys.path.insert(0, str(src_dir))

# Set environment for dev testing
os.environ['REPORT_ENV'] = 'dev'

In [ ]:
import src.silver.customer_dim.fetch_data
import cdutils.deduplication # type: ignore
import cdutils.customer_dim # type: ignore
import cdutils.input_cleansing # type: ignore
import pandas as pd
import src.config
from deltalake import DeltaTable

def sort_dedupe_raw(df, dedupe_list):
    if 'adddate' in df.columns:
        df_sorted = df.sort_values(by='adddate', ascending=False)
    else:
        pass

    result = cdutils.deduplication.dedupe(dedupe_list)
    return result


def generate_customer_dim_table():
    """
    Create Customer Dim table with P+persnbr or O+orgnbr as primary key
    """
    # data = src.silver.customer_dim.fetch_data.fetch_data()
    # wh_pers = data['wh_pers'].copy()
    # wh_org =data['wh_org'].copy()

    wh_pers = DeltaTable(src.config.BRONZE / "wh_pers").to_pandas()
    wh_org = DeltaTable(src.config.BRONZE / "wh_org").to_pandas()

    # Sort descending and dedupe primary keys 
    dedupe_list = [
            {'df':wh_org, 'field':'orgnbr'}
        ]
    wh_org = sort_dedupe_raw(wh_org, dedupe_list)

    dedupe_list = [
            {'df':wh_pers, 'field':'persnbr'}
        ]
    wh_pers = sort_dedupe_raw(wh_pers, dedupe_list)

    # Org + Persify
    wh_org = cdutils.customer_dim.orgify(wh_org, 'orgnbr')
    wh_pers = cdutils.customer_dim.persify(wh_pers, 'persnbr')

    # Concat fields
    customer_dim = pd.concat([wh_org, wh_pers], ignore_index=True)
    return customer_dim

In [ ]:
df = generate_customer_dim_table()

In [ ]:
df